Note: requires tesseract (impira/layoutlm-document-qa) & poppler (pdf2image) to be installed

In [1]:
import os
data = "../../data/datasets/pdf_tressl"
os.listdir(data)[0]

PDF_path = f"{data}/{os.listdir(data)[0]}"

PDF_path

'../../data/datasets/pdf_tressl/AddendumDetail-Report-202311230750 (2023-11-23).pdf'

Seperate each page

In [2]:
import PyPDF2
import os

from marker.convert import convert_single_pdf
from marker.models import load_all_models
import json

fname = PDF_path
model_lst = load_all_models()

print("models loaded")

from vector_store import VectorStore

vs = VectorStore()

def marker(pdf_path):
    full_text, out_meta = convert_single_pdf(pdf_path, model_lst, max_pages=None, parallel_factor=2)

    out_path = f'{pdf_path}_marked.md'

    with open(out_path, "w+", encoding='utf-8') as f:
        f.write(full_text)

    out_meta_filename = out_path.rsplit(".", 1)[0] + "_meta.json"
    with open(out_meta_filename, "w+") as f:
        f.write(json.dumps(out_meta, indent=4))

    return full_text

def split_pdf_into_pages(pdf_path, output_folder):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for page_num in range(len(reader.pages)):
            writer = PyPDF2.PdfWriter()
            writer.add_page(reader.pages[page_num])

            output_filename = os.path.join(output_folder, f'page_{page_num + 1}.pdf')
            with open(output_filename, 'wb') as output_file:
                writer.write(output_file)

            marked_text = marker(output_filename)
            vs.store_page(marked_text, page_num + 1)
            print(f"Ingested: {output_filename}")

split_pdf_into_pages(PDF_path, 'temp')


/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded texify model to mps with torch.float16 dtype
models loaded


/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py:942: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Ingested: temp/page_1.pdf


/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py:942: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Ingested: temp/page_2.pdf
Ingested: temp/page_3.pdf
Ingested: temp/page_4.pdf
Ingested: temp/page_5.pdf
Ingested: temp/page_6.pdf
Ingested: temp/page_7.pdf
Ingested: temp/page_8.pdf
Ingested: temp/page_9.pdf
Ingested: temp/page_10.pdf
Ingested: temp/page_11.pdf
Ingested: temp/page_12.pdf
Ingested: temp/page_13.pdf
Ingested: temp/page_14.pdf
Ingested: temp/page_15.pdf
Ingested: temp/page_16.pdf
Ingested: temp/page_17.pdf
Ingested: temp/page_18.pdf
Ingested: temp/page_19.pdf
Ingested: temp/page_20.pdf
Ingested: temp/page_21.pdf
Ingested: temp/page_22.pdf
Ingested: temp/page_23.pdf
Ingested: temp/page_24.pdf
Ingested: temp/page_25.pdf


In [2]:
from pdf2image import convert_from_path
from PIL import Image

In [3]:
def pdf_to_image(pdf_path):
    images = convert_from_path(pdf_path)

    # Calculate the total height of all images (assuming they have the same width)
    total_height = sum(img.size[1] for img in images)
    max_width = max(img.size[0] for img in images)

    # Create a new image with the appropriate height and width
    combined_image = Image.new('RGB', (max_width, total_height))

    # Paste the images together
    y_offset = 0
    for img in images:
        combined_image.paste(img, (0, y_offset))
        y_offset += img.size[1]

    return combined_image

In [7]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

# nlp = pipeline(
#     "document-question-answering", 
#     model="impira/layoutlm-invoices"
# )

query = "What is the length used by BRIDGE (LOC 131363) - S00132?"
# res_obj = vs.query(query)

# res_obj[0][0]

In [12]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("document-question-answering", model="naver-clova-ix/donut-base-finetuned-docvqa")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.9/site-packages/transformers/models/donut/feature_extraction_donut.py:28: FutureWarning: The class DonutFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DonutImageProcessor instead.
  warnings.warn(


In [18]:
pipe("mistral_paper.png", "What is the vocab_size?")

[{'answer': '32000'}]

In [13]:
img = pdf_to_image(PDF_path)

ans = nlp(
    img,
    query
)

TesseractError: (1, 'Image too large: (1701, 54925) Error during processing.')

In [9]:
print(ans[0]['answer'])

0.01 km
